In [1]:
import Robogame as rg
import networkx as nx
import altair as alt
import time, json
import pandas as pd
import numpy as np
import sklearn.decomposition as skd
from sklearn.linear_model import LinearRegression

To start a game server go into the server directory. Make sure you have flask and networkx installed (```conda install networkx flask flask-cors requests scipy``` should do it).

To start a server do:

```python api.py [-d directoryforgame] [-s] [-t1s T1Secret] [-t2s T2Secret] gameid```

gameid is the the prefix of all the game files (the examples we gave you are examplematch1, examplematch2, examplematch3.

```
-d/--directory is an optional directory. For example, we put the examplematch1 files in the example1 directory
-s/--simulated tells us whether to simulate team 2
-t1s/--team1secret is Team 1's secret. If you don't specify this, the server will give you one
-t2s/--team1secret is Team 2's secret. This is ignored if you use -s. If you don't specify this, the server will give you one
-m/--matchsave is a log file to save the game in, a random log name will be used if you don't specify this
```

Here's a simple game:

```python api.py -d ./example1 -s -t1s bob examplematch1```

We're using the examplematch1 files in the example1 directory, simulating player 2 and team 1's secret is 'bob' (that's us)

In [2]:
# create a game connection using bob as the "secret" key for your team (this is what you're given by the server)
game = rg.Robogame("bob")

# the default is to the localhost on port 5000, if you want something else you can do:
# game = rg.Robogame("bob",server="12.12.31.1",port=2324)

In [3]:
# tell the server we're ready to go
game.setReady()

{'Result': 'OK'}

In [4]:
game.getGameTime()

{'curtime': 0,
 'gameendtime_secs': 1670879437.6351845,
 'gamestarttime_secs': 1670878837.6351845,
 'servertime_secs': 1670878829.7424812,
 'unitsleft': 100}

In [5]:
while(True):
    gametime = game.getGameTime()
    timetogo = gametime['gamestarttime_secs'] - gametime['servertime_secs']
    
    if ('Error' in gametime):
        print("Error"+str(gametime))
        break
    if (timetogo <= 0):
        print("Let's go!")
        break
        
    print("waiting to launch... game will start in " + str(int(timetogo)))
    time.sleep(1) # sleep 1 second at a time, wait for the game to start
    

waiting to launch... game will start in 5
waiting to launch... game will start in 2
Let's go!


In [6]:
for i in np.arange(0,3):
    # this will give us just the new hints, but the object will store everything we've seen
    hints = game.getHints()
    if ('parts' in hints):
        print('we got '+str(len(hints['parts']))+' part hints')
    time.sleep(6)


# get all the hints we've collected
predHints = game.getAllPredictionHints()
partHints = game.getAllPartHints()

we got 0 part hints
we got 12 part hints
we got 12 part hints


# Visu

In [7]:
def familyRelat():
	c1 = pd.DataFrame(game.getTree())
	check = False
	g_id = 1
	c_g_new = pd.DataFrame(columns=['id','id_'+str(g_id),'children'])
	for i in range(len(c1)):
		a = c1['children'][i]
		if 'children' in a:
			children = a['children']
			check = True
		else:
			children = None
		c_g_new = c_g_new.append({'id': c1['id'][i], 'id_'+str(g_id): a['id'], 'children': children},ignore_index=True)
	g_id += 1 
	c1 = c_g_new
	while(check == True):
		check = False
		c_g_new = pd.DataFrame(columns=['id_'+str(g_id-1),'id_'+str(g_id),'children'])
		for i in range(len(c1)):
			a = c1['children'][i]
			for j in range(len(a)):
				b = a[j]
				if 'children' in b:
					children = b['children']
					if len(children)>1:
						check = True
				else:
					children = [{'id': b['id']}]
				c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
		c1 = c1.drop(columns='children').merge(c_g_new,on='id_'+str(g_id-1),how='outer')  
		g_id += 1
	return c1.drop(columns='children')

In [23]:
def FriendGame(id_check,c1):   
	predict = pd.DataFrame(game.getAllPredictionHints())
	predict = predict.rename(columns={'id':'id_data'})
	c1['id'] = c1['id_9']
	Friendship_df = pd.DataFrame(columns=['id_data','G_diff','time','value','id'])
	for i in range(1,len(c1.columns)):
		Friendship = pd.merge(predict,c1,left_on='id_data',right_on='id_'+str(i),how='inner')
		Friendship['G_diff'] = len(c1.columns)-i-1
		Friendship_df = pd.concat([Friendship_df,Friendship[['id_data','G_diff','time','value','id']]])
    
	Friendship_df_app = Friendship_df[Friendship_df['id'].isin(id_check)]
	vis = alt.Chart(Friendship_df_app).mark_circle().encode(
		x = alt.X('time:Q'),
		y = alt.Y("value:Q"),
		color = alt.Color('id:N'),
		opacity = alt.Opacity('G_diff:O'),
		tooltip = alt.Tooltip(['id','time','value']))
	return vis

In [9]:
c1 = familyRelat()

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id': c1['id'][i], 'id_'+str(g_id): a['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id': c1['id'][i], 'id_'+str(g_id): a['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_i

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_i

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_i

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_i

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_i

C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1732\3267116896.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_i

In [41]:
id_check=[10,5,99]
vis = FriendGame(id_check,c1)
vis.interactive()

C:\Users\hp\anaconda3\envs\si649f22\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [40]:
interest_list = [5,10]
game.setRobotInterest(interest_list)

{'Result': 'OK'}

In [ ]:
guess = ({0:99,5:72})
game.setBets(guess)